*This Jupyter Notebook was created by Manuel Klein and belongs to the final project of the Data Science Bootcamp from neuefische Hamburg.*

# US Flight Delay Analysis and Prediction
## Part 1: Data Understanding and Cleaning

This is the first of x Jupyter Notebooks dealing with US flight delays. It covers the understanding of the data, several optimizations and an in-depth analysis and cleaning.

### Table of Contents

<b>[1. Introduction](#anchor_11)</b>
> [1.1 Task](#anchor_11)<br>
> [1.2 Dataset Information](#anchor_12)<br>


<b>[2. Initial Steps](#anchor_21)</b>
> [2.1 Adjusting Jupyter Notebook Settings](#anchor_21)<br>
> [2.2 Importing necessary libraries](#anchor_22)<br>
> [2.3 Importing the data](#anchor_23)

<b>[3. Understanding and optimizing the features](#anchor_31)</b>
> [3.1 Merge check](#anchor_31)<br>
> [3.2 Basic understanding](#anchor_32)<br>
> [3.3 Feature renaming and rearrangement](#anchor_33)<br>
> [3.4 Adding airline names](#anchor_34)<br>
> [3.5 Revision summary](#anchor_35)

<b>[4. Adding airport data](#anchor_41)</b>
> [4.1 Basic data understanding](#anchor_41)<br>
> [4.2 Airport coverage check](#anchor_42)<br>
> [4.3 Removing unnecessary features](#anchor_43)<br>
> [4.4 Missing data overview](#anchor_44)<br>
> [4.5 Adding airport data to original data](#anchor_45)<br>
> [4.6 Missing data relevance check](#anchor_46)<br>
> [4.7 Revision summary](#anchor_47)

<b>[5. Detailed data check and cleanup](#anchor_5)</b>
> [5.1 Removing cancelled and diverted flights](#anchor_51)<br>
> [5.2 Analyzing and reducing missing values](#anchor_52)<br>
> [5.3 Plausibility check](#anchor_53)<br>
> [5.4 Removing non-plausible data](#anchor_54)<br>
> [5.5 Converting flight date format](#anchor_55)<br>
> [5.6 Adding day of week feature](#anchor_56)<br>
> [5.7 Final data check and feature summary](#anchor_57)

<br><br>

<a id='anchor_11'></a>

## 1. Introduction

### 1.1 Task

This project aims at analyzing airline delays and identifying the main influencing factors for aircraft delays. Moreover, a predictive model shall give the opportunity to estimate the likelyhood for a delay of a specific future flight. The following business cases are addressed:

* Airlines need to have a deep understanding of delays for scheduling flights, planning reserve planes at specific airports etc. This includes reasons for delays as well as expected delay times.
* Airlines want to reduce their delays. Thus they need to understand what are the main influencing factors for delays and which of them can be tackled
* Airlines want to understand if the perform better or worse in comparison to their competitors regarding delay at e.g. a specific flight route
* Passengers want to know how likely a certain delay for a specific booking is (e.g. departure and landing airports, date, day of the week, time etc.) and how high that delay might be

<a id='anchor_12'></a>

### 1.2 Dataset Information

The dataset was taken from the following website (downloaded on 12/09/2019):<br>
https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018

The data originally comes from the United States Department of Transportation:<br>
https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time

Airport information data was taken from the following website (downloaded on 12/10/2019):<br>
https://openflights.org/data.html<br>
Necessary data about airports not included in the data file from this website was added manually with information from various internet sources.

<a id='anchor_21'></a>

## 2. Initial Steps

### 2.1 Adjusting Jupyter Notebook Settings

In [1]:
# Adjusting the Jupyter Notebook window width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
# Markdown tables left alignment and markdown table cell content left aligment:

In [3]:
%%html
<style> 
table {float:left}
table td, table th, table tr {text-align:left !important;}
</style>

<a id='anchor_22'></a>

### 2.2 Importing necessary libraries

In [ ]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import datetime
import missingno as msno
import pytz
from datetime import datetime
from pytz import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
# Setting pandas display options to have more columns shown in .head() etc.
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 400)

<a id='anchor_23'></a>

### 2.3 Importing the data

The needed data is imported for each year separately for separate analysis and computation. All data is also stored in a single dataframe for overall analysis.

In [ ]:
# Importing the data for 2016
df16 = pd.read_csv(r'C:\Project_Data_NF/2016.csv')

In [ ]:
# Importing the data for 2017
df17 = pd.read_csv(r'C:\Project_Data_NF/2017.csv')

In [ ]:
# Importing the data for 2018
df18 = pd.read_csv(r'C:\Project_Data_NF/2018.csv')

In [ ]:
# Merging the data into one dataframe
df = pd.concat([df16, df17, df18], ignore_index=True)

In [ ]:
airports = pd.read_csv(r'C:\Project_Data_NF/airports_worldwide.csv')

<a id='anchor_31'></a>

## 3. Understanding the data

### 3.1 Merge check

In [ ]:
# Checking if the number of observations in df is the sum of observations in df16, df17 and df18
len(df16)+len(df17)+len(df18) == len(df)

In [ ]:
# Checking if all .csv-Files contain the same features and the same feature format
print(df16.info())
print(df17.info())
print(df18.info())
print(df.info())

**`➜ All .csv-Files contain the same features and feature formats.`**<br>
**`➜ Merging the dataframes was successful. In the following only the merged dataset will be used as data for single years can easily be filtered from it.`**

In [ ]:
# Reducing RAM load by removing no longer needed data
df16 = df17 = df18 = []

<a id='anchor_32'></a>

### 3.2 Basic understanding

**Feature description from the dataset website:**

| Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | FL_DATE | Date of the flight, yy/mm/dd |
| 2 | OP_CARRIER | Airline Identifier |
| 3 | OP_CARRIER_FL_NUM | Flight Number |
| 4 | ORIGIN | Starting Airport IATA-Code |
| 5 | DEST | Destination Airport IATA-Code |
| 6 | CRS_DEP_TIME | Planned Departure Time |
| 7 | DEP_TIME | Actual Departure Time |
| 8 | DEP_DELAY | Total Delay on Departure in minutes |
| 9 | TAXI_OUT | The time duration elapsed between departure from the origin airport gate and wheels off |
| 10 | WHEELS_OFF | The time point that the aircraft's wheels leave the ground |
| 11 | WHEELS_ON | The time point that the aircraft's wheels touch on the ground |
| 12 | TAXI_IN | The time duration elapsed between wheels-on and gate arrival at the destination airport |
| 13 | CRS_ARR_TIME | Planned arrival time |
| 14 | ARR_TIME | Actual Arrival Time |
| 15 | ARR_DELAY | Total Delay on Arrival in minutes |
| 16 | CANCELLED | Flight Cancelled (0 = not cancelled, 1 = cancelled) |
| 17 | CANCELLATION_CODE | Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security |
| 18 | DIVERTED | Aircraft landed on airport that out of schedule |
| 19 | CRS_ELAPSED_TIME | Planned time amount needed for the flight trip |
| 20 | ACTUAL_ELAPSED_TIME | AIR_TIME+TAXI_IN+TAXI_OUT |
| 21 | AIR_TIME | The time duration between wheels_off and wheels_on time |
| 22 | DISTANCE | Distance between two airports |
| 23 | CARRIER_DELAY | Delay caused by the airline in minutes |
| 24 | WEATHER_DELAY | Delay caused by weather |
| 25 | NAS_DELAY | Delay caused by air system |
| 26 | SECURITY_DELAY | Delay caused by security |
| 27 | LATE_AIRCRAFT_DELAY | Delay caused by aircraft |
| 28 | Unnamed: 27 | Useless column |

**Understanding aircraft delays:**<br><br>
Source: https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

| Delay Reason | Description |
| :-: | :-: |
| Air Carrier | The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, aircraft cleaning, baggage loading, fueling, etc.). |
| Extreme Weather | Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane. |
| National Aviation System (NAS) | Delays and cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control. |
| Late-arriving aircraft | A previous flight with same aircraft arrived late, causing the present flight to depart late. |
| Security | Delays or cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas. |



In [ ]:
# First impression of the data
df.head(3)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
# Deleting the last column ('Unnamed: 27') in the Dataset as it is a useless column according to the feature description
df = df.iloc[:,:27]

<a id='anchor_33'></a>

### 3.3 Feature renaming and rearrangement

In [ ]:
# Renaming columns to make them easier understandable
column_renaming_dict = {"OP_CARRIER": "AIRLINE_ID",
                        "OP_CARRIER_FL_NUM": "FL_NUMBER",
                        "ORIGIN": "DEP_AIRPORT",
                        "DEST": "ARR_AIRPORT",
                        "CRS_DEP_TIME": "DEP_TIME_PLANNED",
                        "TAXI_OUT": "TAXI_OUT_DURATION",
                        "WHEELS_OFF": "WHEELS_OFF_TIME",
                        "WHEELS_ON": "WHEELS_ON_TIME",
                        "TAXI_IN": "TAXI_IN_DURATION",
                        "CRS_ARR_TIME": "ARR_TIME_PLANNED",
                        "CRS_ELAPSED_TIME": "TRAVEL_DURATION_PLANNED",
                        "ACTUAL_ELAPSED_TIME": "TRAVEL_DURATION",
                        "AIR_TIME": "IN_AIR_DURATION"}

df.rename(columns=column_renaming_dict, inplace=True)

In [ ]:
# Optimizing the order of features in the dataframes for better overview
optimized_column_order = ["FL_DATE", "AIRLINE_ID", "FL_NUMBER", "DEP_AIRPORT", "ARR_AIRPORT", "DISTANCE", "DEP_TIME_PLANNED",
                          "DEP_TIME", "DEP_DELAY", "TAXI_OUT_DURATION", "WHEELS_OFF_TIME", "WHEELS_ON_TIME", "IN_AIR_DURATION", "TAXI_IN_DURATION",
                          "ARR_TIME_PLANNED", "ARR_TIME", "ARR_DELAY", "TRAVEL_DURATION_PLANNED", "TRAVEL_DURATION", "DIVERTED", "CANCELLED",
                          "CANCELLATION_CODE", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]

df = df.reindex(columns=optimized_column_order)

<a id='anchor_34'></a>

### 3.4 Adding airline names

In [ ]:
# Checking which airline IDs are included in the dataset
sorted(df["AIRLINE_ID"].unique())

In [ ]:
# Understanding the meaning of the airline IDs (IATA codes)
# Data source: https://en.wikipedia.org/wiki/List_of_airline_codes
airline_dict = {'9E': 'Endeavor Air',
                'AA': 'American Airlines',
                'AS': 'Alaska Airlines',
                'B6': 'JetBlue Airways',
                'DL': 'Delta Air Lines',
                'EV': 'ExpressJet Airlines LLC',
                'F9': 'Frontier Airlines',
                'G4': 'Allegiant Air',
                'HA': 'Hawaiian Airlines',
                'MQ': 'Envoy Air',
                'NK': 'Spirit Airlines,',
                'OH': 'PSA Airlines',
                'OO': 'SkyWest Airlines',
                'UA': 'United Air Lines',
                'VX': 'Virgin America',
                'WN': 'Southwest Airlines Co.',
                'YV': 'Mesa Airlines',
                'YX': 'Midwest Airlines'}

In [ ]:
# Creating new column with airline names
df['AIRLINE_NAME'] = df['AIRLINE_ID'].values
df['AIRLINE_NAME'] = df['AIRLINE_NAME'].map(airline_dict)

In [ ]:
# Checking the index of the newly created column
df.columns.get_loc("AIRLINE_NAME")

In [ ]:
# Moving the new created airline name column next to the airline ID column
cols = df.columns.tolist()
cols = cols[0:2] + [cols[27]] + cols[2:27]
df = df.reindex(columns = cols)

<a id='anchor_35'></a>

### 3.5 Revision summary

In [ ]:
# Check if all steps described above were successfully performed
df.head(1)

**Feature explanation after revision:**

Features containing time information refer to actual time except their name contains the key word "planned".<br>
Features containing the keyword "time" always describe a point in time. Otherwise the word "duration" is used.


|Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | FL_DATE | Date of the flight, yy/mm/dd |
| 2 | AIRLINE_ID | Airline Identifier |
| 3 | AIRLINE_NAME | Name of the airline |
| 4 | FL_NUMBER | Flight Number |
| 5 | DEP_AIRPORT | Departure Airport IATA-Code |
| 6 | ARR_AIRPORT | Arrival Airport IATA-Code |
| 7 | DISTANCE | Distance between departure airport and arrival airport in miles |
| 8 | DEP_TIME_PLANNED | Planned Departure Time |
| 9 | DEP_TIME | Actual Departure Time |
| 10 | DEP_DELAY | Total Delay on Departure in minutes |
| 11 | TAXI_OUT_DURATION | The time duration elapsed between departure from the departure airport gate and wheels off |
| 12 | WHEELS_OFF_TIME | The time point that the aircraft's wheels leave the ground |
| 13 | WHEELS_ON_TIME | The time point that the aircraft's wheels touch on the ground |
| 14 | IN_AIR_DURATION | The time duration between wheels_off and wheels_on time |
| 15 | TAXI_IN_DURATION | The time duration elapsed between wheels-on and gate arrival at the arrival airport |
| 16 | ARR_TIME_PLANNED | Planned arrival time |
| 17 | ARR_TIME | Actual arrival Time |
| 18 | ARR_DELAY | Total delay on arrival in minutes |
| 19 | TRAVEL_DURATION_PLANNED | Planned time amount needed for the flight trip |
| 20 | TRAVEL_DURATION | Actual time amount needed for the flight trip |
| 21 | DIVERTED | Aircraft landed on a different airport than planned (0 = no, 1 = yes) |
| 22 | CANCELLED | Flight Cancelled (0 = not cancelled, 1 = cancelled) |
| 23 | CANCELLATION_CODE | Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security |
| 24 | CARRIER_DELAY | Delay caused by the airline in minutes |
| 25 | WEATHER_DELAY | Delay caused by weather |
| 26 | NAS_DELAY | Delay caused by air system |
| 27 | SECURITY_DELAY | Delay caused by security |
| 28 | LATE_AIRCRAFT_DELAY | Delay caused by aircraft |

<a id='anchor_41'></a>

## 4. Adding airport data

### 4.1 Basic data understanding

In [ ]:
airports.head()

In [ ]:
# Checking the number of US airports for each time zone. '\N' means 'NaN'
airports[airports['Country'] == "United States"]['Timezone'].value_counts().sort_index(ascending=True)

| Offset | Timezone | Abbreviation | Name | Example City |
| :-: | :-: | :-: | :-: | :-: |
| UTC | -10 | HST | Hawaii Standard Time | Honolulu |
| UTC | -9 | AKST | Alaska Standard Time | Anchorage |
| UTC | -8 | PST | Pacific Standard Time | Los Angeles |
| UTC | -7 | MST | Mountain Standard Time | Salt Lake City |
| UTC | -6 | CST | Central Standard Time | Chicago |
| UTC | -5 | ES | Eastern Standard Time | New York |

<a id='anchor_42'></a>

### 4.2 Airport coverage check

In [ ]:
a1 = df["DEP_AIRPORT"].unique()
a2 = df["ARR_AIRPORT"].unique()

In [ ]:
b = airports["IATA"].unique()

In [ ]:
# Checking if all airports mentioned in the flights dataset are included in the airport dataset
# Return the unique values in a that are not in b
print(np.setdiff1d(a1, b))
print(np.setdiff1d(a2, b))

**`➜ All airports mentioned in the flights dataset are included in the airport dataset.`**<br>
**`➜ However, filtering the country for "United States" in advance leads to airport information missing.`**<br>
**`This means that some airports of the flight dataset are outside the United States!`**<br>

<a id='anchor_43'></a>

### 4.3 Removing unnecessary features

In [ ]:
# Deleting columns that are not needed for this project
airports.drop(columns=['Airport_ID','Country','Type','Source', 'ICAO','DST'], inplace=True)
#airports.drop(columns=['Airport_ID','Country','ICAO','Timezone','DST','Tz_database_time_zone','Type','Source'], inplace=True)

In [ ]:
# Checking the dataset after deletion of unnecessary columns
airports.head(3)

<a id='anchor_44'></a>

### 4.4 Missing data overview

In [ ]:
# Replacing '\N' entries in the dataset by NaN values, enabling missing data analysis
airports = airports.replace(r'\N', np.nan)

In [ ]:
# Checking the number of NaN values for each feature inside the airport dataset
airports.isna().sum()

In [ ]:
# Checking the percentage of NaN values for each feature inside the airport dataset
airports.isna().sum() / len(airports)

<a id='anchor_45'></a>

### 4.5 Adding airport data to original data

In [ ]:
# Preparing departure airport data for merging
dep_airports = airports.copy()
dep_airports = dep_airports.rename(columns={"Name": "DEP_AIRPORT_NAME",
                                            "City": "DEP_CITY",
                                            "IATA": "DEP_AIRPORT",
                                            "Latitude":"DEP_LAT",
                                            "Longitude": "DEP_LONG",
                                            "Altitude": "DEP_ALT",
                                            "Timezone": "DEP_UTC",
                                            "Tz_database_time_zone": "DEP_TZ"})

In [ ]:
# Preparing arrival airport data for merging
arr_airports = airports.copy()
arr_airports = arr_airports.rename(columns={"Name": "ARR_AIRPORT_NAME",
                                            "City": "ARR_CITY",
                                            "IATA": "ARR_AIRPORT",
                                            "Latitude":"ARR_LAT",
                                            "Longitude": "ARR_LONG",
                                            "Altitude": "ARR_ALT",
                                            "Timezone": "ARR_UTC",
                                            "Tz_database_time_zone": "ARR_TZ"})

In [ ]:
dep_airports.head(3)

In [ ]:
arr_airports.head(3)

In [ ]:
# Merging departure airport data to original dataframe
df = pd.merge(df, dep_airports, on='DEP_AIRPORT')

In [ ]:
# Merging arrival airport data to original dataframe
df = pd.merge(df, arr_airports, on='ARR_AIRPORT')

<a id='anchor_46'></a>

### 4.6 Missing data relevance check

In [ ]:
# Creating a new dataframe containing all features that have been added to the original dataframe
df_new = df[['DEP_AIRPORT_NAME', 'DEP_CITY','DEP_LAT','DEP_LONG','DEP_ALT', 'DEP_UTC', 'DEP_TZ', 'ARR_AIRPORT_NAME', 'ARR_CITY','ARR_LAT','ARR_LONG','ARR_ALT', 'ARR_UTC', 'ARR_TZ']]

In [ ]:
# Checking if any of the NaN values from the airport dataset are relevant for the flights dataset
df_new.isna().sum()

**`➜ Luckily no NaN values from the airport dataset are relevant for the flights dataset at hand.`**<br>

In [ ]:
# Reducing RAM load by removing no longer needed data
df_new = []

<a id='anchor_47'></a>

### 4.7 Revision summary

In [ ]:
df.head(3)

**Feature explanation after including airport data:**

Features containing time information refer to actual time except their name contains the key word "planned".<br>
Features containing the keyword "time" always describe a point in time. Otherwise the word "duration" is used.


|Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | FL_DATE | Date of the flight, yyyy-mm-dd |
| 2 | AIRLINE_ID | Airline Identifier |
| 3 | AIRLINE_NAME | Name of the airline |
| 4 | FL_NUMBER | Flight Number |
| 5 | DEP_AIRPORT | Departure Airport IATA-Code |
| 6 | ARR_AIRPORT | Arrival Airport IATA-Code |
| 7 | DISTANCE | Distance between departure airport and arrival airport in miles |
| 8 | DEP_TIME_PLANNED | Planned Departure Time |
| 9 | DEP_TIME | Actual Departure Time |
| 10 | DEP_DELAY | Total Delay on Departure in minutes |
| 11 | TAXI_OUT_DURATION | The time duration elapsed between departure from the departure airport gate and wheels off in minutes|
| 12 | WHEELS_OFF_TIME | The time point that the aircraft's wheels leave the ground |
| 13 | WHEELS_ON_TIME | The time point that the aircraft's wheels touch on the ground |
| 14 | IN_AIR_DURATION | The time duration between wheels_off and wheels_on time in minutes |
| 15 | TAXI_IN_DURATION | The time duration elapsed between wheels-on and gate arrival at the arrival airport in minutes |
| 16 | ARR_TIME_PLANNED | Planned arrival time |
| 17 | ARR_TIME | Actual arrival Time |
| 18 | ARR_DELAY | Total delay on arrival in minutes |
| 19 | TRAVEL_DURATION_PLANNED | Planned time amount needed for the flight trip in minutes |
| 20 | TRAVEL_DURATION | Actual time amount needed for the flight trip in minutes |
| 21 | DIVERTED | Aircraft landed on a different airport than planned (0 = no, 1 = yes) |
| 22 | CANCELLED | Flight Cancelled (0 = not cancelled, 1 = cancelled) |
| 23 | CANCELLATION_CODE | Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security |
| 24 | CARRIER_DELAY | Delay caused by the airline in minutes |
| 25 | WEATHER_DELAY | Delay caused by weather in minutes |
| 26 | NAS_DELAY | Delay caused by air system in minutes |
| 27 | SECURITY_DELAY | Delay caused by security in minutes |
| 28 | LATE_AIRCRAFT_DELAY | Delay caused by aircraft in minutes|
| 29 | DEP_AIRPORT_NAME | Name of departure airport |
| 30 | DEP_CITY | Name of departure city |
| 31 | DEP_LAT | Latitude of departure airport |
| 32 | DEP_LONG | Longitude of departure airport |
| 33 | DEP_ALT | Altitude of departure airport in ft |
| 34 | DEP_UTC | UTC time of departure airport |
| 35 | DEP_TZ | TZ Olson time of departure airport |
| 36 | ARR_AIRPORT_NAME | Name of arrival airport |
| 37 | ARR_CITY | Name of arrival city |
| 38 | ARR_LAT | Latitude of arrival airport |
| 39 | ARR_LONG | Longitude of arrival airport |
| 40 | ARR_ALT | Altitude of arrival airport in ft |
| 41 | ARR_UTC | UTC time of arrival airport |
| 42 | ARR_TZ | TZ Olson time of arrival airport |

In [ ]:
df.to_pickle(r'C:\Project_Data_NF\dfendofstep4.pkl')

<a id='anchor_51'></a>

<a id='anchor_5'></a>

## 5. Detailed data check and cleanup

To do:
* Times into time format?
* Adding holiday column?

In [ ]:
df = pd.read_pickle(r'C:\Project_Data_NF/dfendofstep4.pkl')

In [ ]:
# Overview of current dataframe
df.head(3)

In [ ]:
df.info()

In [ ]:
df.shape

<a id='anchor_51'></a>

### 5.1 Removing cancelled and diverted flights

Diverted flights contain many values that are not interpretable. The arrival time e.g. is not interpretable as the information to which airport a diverted flight actually went is missing. Keeping this data would even tend to falsify analyses and thus is removed from the dataset.

Cancelled flights contain many NaN values and do not help to predict flight delay. Thus these flights are removed as well. A data analysis for cancelled flights can still be perfomed with the pickled data status at the end of chapter 4.

In [ ]:
# 0.2 percent of all flights have been diverted
df['DIVERTED'].value_counts().values[1] / df['DIVERTED'].value_counts().values[0] * 100

In [ ]:
# 1.45 percent of all flights have been cancelled
df['CANCELLED'].value_counts().values[1] / df['CANCELLED'].value_counts().values[0] * 100

In [ ]:
# 1.67 percent of data is lost by removing all flights from the dataset that are diverted or cancelled
(1 - (len(df[((df['DIVERTED'] == 0) & (df['CANCELLED'] == 0))])) / len(df)) * 100

In [ ]:
# Removing diverted and cancelled flights from the dataset
df = df[((df['DIVERTED'] == 0) & (df['CANCELLED'] == 0))]

In [ ]:
# Resetting the index, so that the last ID is equal to the number of rows in the dataset
# Necessary after rows have been dropped, so that access via index (e.g. .iloc) works correctly
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [ ]:
# Removing columns related to diverted and cancelled flights as the do no longer contain any information
df.drop(columns=['DIVERTED','CANCELLED', 'CANCELLATION_CODE'], inplace=True)

In [ ]:
df.shape

In [ ]:
# Dropping all flights from and to airports with a positive UTC leads to a removal of flight over the data line
# This step is necessary for re-calculation of features
# Dropping all flights from and to airports with a positive UTC leads to a neglectible loss of data (0.01%)
(1 - (len(df[((df['DEP_UTC'].astype('int') < 0) & (df['ARR_UTC'].astype('int') < 0))]) / len(df))) * 100

In [ ]:
# Performing the dropping of all flights from and to airports with a positive UTC
df = df[((df['DEP_UTC'].astype('int') < 0) & (df['ARR_UTC'].astype('int') < 0))]

In [ ]:
# Resetting the index, so that the last ID is equal to the number of rows in the dataset
# Necessary after rows have been dropped, so that access via index (e.g. .iloc) works correctly
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

<a id='anchor_52'></a>

### 5.2 Analyzing and reducing missing values

In [ ]:
# Checking the absolute and relative amount of NaN values for each feature inside the flights dataset
s_missing = df.isna().sum()
s_missing_percentage = df.isna().sum() / len(df) * 100
df_missing = pd.concat([s_missing, s_missing_percentage], axis=1)
df_missing.columns = ['Absolute missing', 'Percentage missing']
df_missing

NaN Values are okay for columns describing delay reasons as not all flights have delays. In this case these columns are empty.<br>
Missing data for 'DEP_DELAY' and 'ARR_DELAY' can be recalculated as follows:
* DEP_DELAY = DEP_TIME - DEP_TIME_PLANNED
* ARR_DELAY = ARR_TIME - ARR_TIME_PLANNED

In [ ]:
df['DEP_DELAY_NEW'] = df[df['DEP_DELAY'].isna()]['DEP_TIME'] - df[df['DEP_DELAY'].isna()]['DEP_TIME_PLANNED']

In [ ]:
df[df['DEP_DELAY_NEW'].notna()]['DEP_DELAY_NEW'].value_counts()

In [ ]:
df['ARR_DELAY_NEW'] = df[df['ARR_DELAY'].isna()]['ARR_TIME'] - df[df['ARR_DELAY'].isna()]['ARR_TIME_PLANNED']

In [ ]:
df[df['ARR_DELAY_NEW'].notna()]['ARR_DELAY_NEW'].value_counts()

It can be seen that all missing values for 'DEP_DELAY' and 'ARR_DELAY' are zero and thus can be filled with zeros.

In [ ]:
# Deleting the columns that were created for analysis purposes
df.drop(columns=['DEP_DELAY_NEW','ARR_DELAY_NEW'], inplace=True)

In [ ]:
# Missing values for 'DEP_DELAY' and 'ARR_DELAY' are filled with zeros
df['DEP_DELAY'].fillna(0, inplace=True)
df['ARR_DELAY'].fillna(0, inplace=True)

In [ ]:
# Dropping columns with neglectable amount of missing data without further analysis
df = df[df['WHEELS_ON_TIME'].notna()]
df = df[df['IN_AIR_DURATION'].notna()]
df = df[df['TAXI_IN_DURATION'].notna()]
df = df[df['ARR_TIME'].notna()]
df = df[df['TRAVEL_DURATION'].notna()]

In [ ]:
# Resetting the index, so that the last ID is equal to the number of rows in the dataset
# Necessary after rows have been dropped, so that access via index (e.g. .iloc) works correctly
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [ ]:
# Check if missing value cleanup was successful
s_missing = df.isna().sum()
s_missing_percentage = df.isna().sum() / len(df) * 100
df_missing = pd.concat([s_missing, s_missing_percentage], axis=1)
df_missing.columns = ['Absolute missing', 'Percentage missing']
df_missing

<a id='anchor_53'></a>

### 5.3 Plausibility check

In [ ]:
df.describe().loc[['min','max']]

Findings:
 * DEP_TIME, WHEELS_OFF_TIME, WHEELS_ON_TIME, ARR_TIME_PLANNED, ARR_TIME: Max value is 2400, which is not compatible with datetime format. 24:00 to be converted to 00:00.
 * DEP_DELAY: Min and max values not plausible. To be clarified.
 * TRAVEL_DURATION_PLANNED: Negative values clearly show wrong calculation
 * With help of the distance, realistic flight durations can be estimated.

<a id='anchor_54'></a>

### 5.4 Removing non-plausible data

In [ ]:
def timeconverter(x):
    if x == 2400:
        x = 0
    return x

In [ ]:
# Converting 24:00 to 00:00
df['DEP_TIME'] = df['DEP_TIME'].apply(timeconverter)
df['WHEELS_OFF_TIME'] = df['WHEELS_OFF_TIME'].apply(timeconverter)
df['WHEELS_ON_TIME'] = df['WHEELS_ON_TIME'].apply(timeconverter)
df['ARR_TIME_PLANNED'] = df['ARR_TIME_PLANNED'].apply(timeconverter)
df['ARR_TIME'] = df['ARR_TIME'].apply(timeconverter)

In [ ]:
# Defining a feature for average travel speed
df['AVERAGE_MPH_PLANNED'] = df['DISTANCE'] / (df['TRAVEL_DURATION_PLANNED'] / 60)

In [ ]:
df.sort_values(by=['AVERAGE_MPH_PLANNED'], ascending=False).head(10)

In [ ]:
# 575 Mph is the expected max. flight speed. As planes are slower during takeoff and landing, even a buffer is included in this calculation.
# 54 flights in the dataset have an average travel speed that is larger than 575 Mph
df[df['AVERAGE_MPH_PLANNED'] > 575].shape

In [ ]:
# Removing flights in the dataset that have an average travel speed that is larger than 575 Mph or negative
df = df[df['AVERAGE_MPH_PLANNED'] < 575]
df = df[df['AVERAGE_MPH_PLANNED'] > 0]

In [ ]:
# Resetting the index, so that the last ID is equal to the number of rows in the dataset
# Necessary after rows have been dropped, so that access via index (e.g. .iloc) works correctly
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [ ]:
# Removing column as no longer needed
df.drop(columns=['AVERAGE_MPH_PLANNED'], inplace=True)

In [ ]:
df.to_pickle(r'C:\Project_Data_NF\dfendofstep54.pkl')

<a id='anchor_55'></a>

### 5.5 Convert features to datetime format

In [ ]:
# Converting 'DEP_TIME_PLANNED' and 'ARR_TIME_PLANNED' to datetime (several intermediate steps necessary)
df['DISTANCE'] = df['DISTANCE'].astype('int64')
df['DEP_TIME_PLANNED_STR'] = df['DEP_TIME_PLANNED'].astype('str')
df['ARR_TIME_PLANNED_STR'] = df['ARR_TIME_PLANNED'].astype('str')

In [ ]:
def zeropadding(x):
    if len(x) == 1:
        x = '000' + x
    elif len(x) == 2:
        x = '00' + x
    elif len(x) == 3:
        x = '0' + x
    if len(x) != 4:
        print(x)
    return x

In [ ]:
df['DEP_TIME_PLANNED_STR'] = df['DEP_TIME_PLANNED_STR'].apply(zeropadding)
df['ARR_TIME_PLANNED_STR'] = df['ARR_TIME_PLANNED_STR'].apply(zeropadding)

In [ ]:
def coloninseration(x):
    x = str(x)
    return x[:-2] + ':' + x[-2:]

In [ ]:
df['DEP_TIME_PLANNED_STR'] = df['DEP_TIME_PLANNED_STR'].apply(coloninseration)
df['ARR_TIME_PLANNED_STR'] = df['ARR_TIME_PLANNED_STR'].apply(coloninseration)

In [ ]:
df['DEP_DATETIME_PLANNED'] = df['FL_DATE'] + ' ' + df['DEP_TIME_PLANNED_STR']
df['ARR_DATETIME_PLANNED'] = df['FL_DATE'] + ' ' + df['ARR_TIME_PLANNED_STR']

In [ ]:
def strtodatetime(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M')

In [ ]:
df['DEP_DATETIME_PLANNED'] = df['DEP_DATETIME_PLANNED'].apply(strtodatetime)
df['ARR_DATETIME_PLANNED'] = df['ARR_DATETIME_PLANNED'].apply(strtodatetime)

In [ ]:
# Removing column as no longer needed
df.drop(columns=['DEP_TIME_PLANNED_STR', 'ARR_TIME_PLANNED_STR'], inplace=True)

In [ ]:
# Creating a new feature, delivering negative values if arrival date is departure date + 1 day
df['OVERMIDNIGHT'] = df['ARR_TIME_PLANNED'] - df['DEP_TIME_PLANNED']

In [ ]:
def overmidnighttobinary(x):
    if x >= 0:
        x = 0
    elif x < 0:
        x = 1
    return x

In [ ]:
# Converting the values of OVERMIDNIGHT to 1 if arrival date is departure date + 1 day, otherwise to 0
df['OVERMIDNIGHT'] = df['OVERMIDNIGHT'].apply(overmidnighttobinary)

In [ ]:
df['ARR_DATETIME_PLANNED2'] = df[df['OVERMIDNIGHT'] == 1]['ARR_DATETIME_PLANNED'] + pd.Timedelta('1 day')

In [ ]:
df.loc[df['ARR_DATETIME_PLANNED2'].isnull(),'ARR_DATETIME_PLANNED2'] = df['ARR_DATETIME_PLANNED']

In [ ]:
df.drop(columns=['ARR_DATETIME_PLANNED'], inplace=True)

In [ ]:
df.rename(columns={"ARR_DATETIME_PLANNED2": "ARR_DATETIME_PLANNED"}, inplace=True)

In [ ]:
df.head(3)

In [ ]:
df[df['OVERMIDNIGHT'] == 1].head()

In [ ]:
df.isna().sum()

In [ ]:
# Converting the feature 'FL_DATE' in date/time format
df['FL_DATE'] = df['FL_DATE'].astype('datetime64[D]')

In [ ]:
# Plausibility check: There should be no day with no flights in the USA between 2016 and 2018 --> OK
from datetime import date, timedelta
date_set = set(df.FL_DATE.min() + timedelta(x) for x in range((df.FL_DATE.max() - df.FL_DATE.min()).days))
missing = sorted(date_set - set(df.FL_DATE))
missing

In [ ]:
df.head()

<a id='anchor_56'></a>

### 5.6 Adding day of week feature

In [ ]:
# Add column for day of week (Monday=0, Sunday=6)
df['FL_DAYOFWEEK'] = df['FL_DATE'].dt.dayofweek

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
# Moving day of week column next to the FL_DATE column
cols = df.columns.tolist()
cols = cols[0:1] + [cols[42]] + cols[1:42]
df = df.reindex(columns = cols)

<a id='anchor_57'></a>

### 5.7 Final data check and feature summary

In [ ]:
df.head(3)

**Feature explanation after data cleaning:**

Features containing time information refer to actual time except their name contains the key word "planned".<br>
Features containing the keyword "time" always describe a point in time. Otherwise the word "duration" is used.


|Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | FL_DATE | Date of the flight, yyyy-mm-dd |
| 2 | FL_DAYOFWEEK | Day of week of the flight (Monday=0, Sunday=6) |
| 3 | AIRLINE_ID | Airline Identifier |
| 4 | AIRLINE_NAME | Name of the airline |
| 5 | FL_NUMBER | Flight Number |
| 6 | DEP_AIRPORT | Departure Airport IATA-Code |
| 7 | ARR_AIRPORT | Arrival Airport IATA-Code |
| 8 | DISTANCE | Distance between departure airport and arrival airport in miles |
| 9 | DEP_TIME_PLANNED | Planned Departure Time |
| 10 | DEP_TIME | Actual Departure Time |
| 11 | DEP_DELAY | Total Delay on Departure in minutes |
| 12 | TAXI_OUT_DURATION | The time duration elapsed between departure from the departure airport gate and wheels off in minutes|
| 13 | WHEELS_OFF_TIME | The time point that the aircraft's wheels leave the ground |
| 14 | WHEELS_ON_TIME | The time point that the aircraft's wheels touch on the ground |
| 15 | IN_AIR_DURATION | The time duration between wheels_off and wheels_on time in minutes |
| 16 | TAXI_IN_DURATION | The time duration elapsed between wheels-on and gate arrival at the arrival airport in minutes |
| 17 | ARR_TIME_PLANNED | Planned arrival time |
| 18 | ARR_TIME | Actual arrival Time |
| 19 | ARR_DELAY | Total delay on arrival in minutes |
| 20 | TRAVEL_DURATION_PLANNED | Planned time amount needed for the flight trip in minutes |
| 21 | TRAVEL_DURATION | Actual time amount needed for the flight trip in minutes |
| 22 | CARRIER_DELAY | Delay caused by the airline in minutes |
| 23 | WEATHER_DELAY | Delay caused by weather in minutes |
| 24 | NAS_DELAY | Delay caused by air system in minutes |
| 25 | SECURITY_DELAY | Delay caused by security in minutes |
| 26 | LATE_AIRCRAFT_DELAY | Delay caused by aircraft in minutes|
| 27 | DEP_AIRPORT_NAME | Name of departure airport |
| 28 | DEP_CITY | Name of departure city |
| 29 | DEP_LAT | Latitude of departure airport |
| 30 | DEP_LONG | Longitude of departure airport |
| 31 | DEP_ALT | Altitude of departure airport in ft |
| 32 | DEP_UTC | UTC time of departure airport |
| 33 | DEP_TZ | TZ Olson time of departure airport |
| 34 | ARR_AIRPORT_NAME | Name of arrival airport |
| 35 | ARR_CITY | Name of arrival city |
| 36 | ARR_LAT | Latitude of arrival airport |
| 37 | ARR_LONG | Longitude of arrival airport |
| 38 | ARR_ALT | Altitude of arrival airport in ft |
| 39 | ARR_UTC | UTC time of arrival airport |
| 40 | ARR_TZ | TZ Olson time of arrival airport |
| 41 | DEP_DATETIME_PLANNED | UTC time of arrival airport |
| 42 | OVERMIDNIGHT | 1 if flight is over midnight, 0 if not |
| 43 | ARR_DATETIME_PLANNED | TZ Olson time of arrival airport |

In [ ]:
df.to_pickle(r'C:\Project_Data_NF\dfendofstep5.pkl')

## No longer needed

**Tackling identified issues**:

Estimating trustworthy features:
* DEP_TIME_PLANNED
* ARR_TIME_PLANNED
* DEP_UTC (airports in Arizona like Phoenix to be cross-checked)
* ARR_UTC (airports in Arizona like Phoenix to be cross-checked)<br>
--> TRAVEL_DURATION_PLANNED to be re-calculated. Deviations to be analyzed.

* WHEELS_OFF_TIME
* WHEELS_ON_TIME<br>
--> IN_AIR_DURATION to be re-calculated. Deviations to be analyzed.

* TAXI_OUT_DURATION
* TAXI_IN_DURATION<br>
--> Together with the re-calculated IN_AIR_DURATION, the TRAVEL_DURATION is re-calculated. Deviations to be analyzed.
--> Together with the re-calculated TRAVEL_DURATION, the ARR_TIME is re-calculated. Deviations to be analyzed.
--> Together with the re-calculated ARR_TIME, the ARR_DELAY is re-calculated. Deviations to be analyzed.



In [ ]:
# Creating a new feature, delivering negative values if arrival date is departure date + 1 day
df['OVERMIDNIGHT'] = df['ARR_TIME_PLANNED'] - df['DEP_TIME_PLANNED']

In [ ]:
def overmidnighttobinary(x):
    if x >= 0:
        x = 0
    elif x < 0:
        x = 1
    return x

In [ ]:
# Converting the values of OVERMIDNIGHT to 1 if arrival date is departure date + 1 day, otherwise to 0
df['OVERMIDNIGHT'] = df['OVERMIDNIGHT'].apply(overmidnighttobinary)

In [ ]:
df.head()

In [ ]:
# Checking the number of NaN values for each feature inside the flights dataset
df.isna().sum()

In [ ]:
# Converting float to int format
df['DISTANCE'] = df['DISTANCE'].astype('int64')
df['DEP_TIME_PLANNED'] = df['DEP_TIME_PLANNED'].astype('str')
df['ARR_TIME_PLANNED'] = df['ARR_TIME_PLANNED'].astype('str')

In [ ]:
def zeropadding(x):
    if len(x) == 1:
        x = '000' + x
    elif len(x) == 2:
        x = '00' + x
    elif len(x) == 3:
        x = '0' + x
    if len(x) != 4:
        print(x)
    return x

In [ ]:
df['DEP_TIME_PLANNED'] = df['DEP_TIME_PLANNED'].apply(zeropadding)
df['ARR_TIME_PLANNED'] = df['ARR_TIME_PLANNED'].apply(zeropadding)

In [ ]:
df.head(150)

In [ ]:
def coloninseration(x):
    x = str(x)
    return x[:-2] + ':' + x[-2:]

In [ ]:
df['DEP_TIME_PLANNED'] = df['DEP_TIME_PLANNED'].apply(coloninseration)
df['ARR_TIME_PLANNED'] = df['ARR_TIME_PLANNED'].apply(coloninseration)

In [ ]:
df['DEP_DATETIME_PLANNED'] = df['FL_DATE'] + ' ' + df['DEP_TIME_PLANNED']
df['ARR_DATETIME_PLANNED'] = df['FL_DATE'] + ' ' + df['ARR_TIME_PLANNED']

In [ ]:
def strtodatetime(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M')

In [ ]:
df['DEP_DATETIME_PLANNED'] = df['DEP_DATETIME_PLANNED'].apply(strtodatetime)
df['ARR_DATETIME_PLANNED'] = df['ARR_DATETIME_PLANNED'].apply(strtodatetime)

In [ ]:
df.isna().sum()

In [ ]:
df['ARR_DATETIME_PLANNED2'] = df[df['OVERMIDNIGHT'] == 1]['ARR_DATETIME_PLANNED'] + pd.Timedelta('1 day')

In [ ]:
df.loc[df['ARR_DATETIME_PLANNED2'].isnull(),'ARR_DATETIME_PLANNED2'] = df['ARR_DATETIME_PLANNED']

In [ ]:
df.drop(columns=['ARR_DATETIME_PLANNED'], inplace=True)

In [ ]:
df.rename(columns={"ARR_DATETIME_PLANNED2": "ARR_DATETIME_PLANNED"}, inplace=True)

In [ ]:
df.head(3)

In [ ]:
df[df['OVERMIDNIGHT'] == 1].head()

In [ ]:
df.isna().sum()

In [ ]:
df = pd.read_pickle(r'C:\Project_Data_NF/dfendofstep5.pkl')

It can be seen that the number of missing values for 'ARR_DELAY' is higher than the number of missing values for 'ARR_TIME'. As 'ARR_TIME_PLANNED' does not have any missing values, the number of missing values for 'ARR_DELAY' can at least be reduced to the number of missing values for 'ARR_TIME' by calculating these values.

ARR_DELAY = ARR_TIME - ARR_TIME_PLANNED

In [ ]:
df.head()

In [ ]:
#  Recovering ARR_DELAY data from ARR_TIME_PLANNED and ARR_TIME
df.info()

In [ ]:
df.isna().sum()

In [ ]:
def f1(x):
    return int(int(x) % 100 + (int(x) - int(x) % 100) * 0.6)

In [ ]:
df['DEP_TIME_PLANNED_COPY'] = df['DEP_TIME_PLANNED'].apply(f1)

In [ ]:
df['ARR_TIME_PLANNED_COPY'] = df['ARR_TIME_PLANNED'].apply(f1)

In [ ]:
df.tail(30)

In [ ]:
df.tail(50)

In [ ]:
df['TRAVEL_DURATION_PLANNED'] = df['ARR_TIME_PLANNED_COPY'].subtract(df['DEP_TIME_PLANNED_COPY'])

In [ ]:
def f2(x):
    if x<0:
        x = x + (24*60)
    return x

In [ ]:
df['TRAVEL_DURATION_PLANNED2'] = df['TRAVEL_DURATION_PLANNED'].apply(f2)

In [ ]:
df[(df['TRAVEL_DURATION_PLANNED2'] <=5)].head()

In [ ]:
df[(df['ARR_DELAY'].isnull()) & (df['ARR_TIME'] < 300)].shape

In [ ]:
# Checking the percentage of NaN values for each feature inside the flights dataset
df.isna().sum() / len(df)

NaN Values are okay for columns describing delay reasons as not all flights have delays. In this case these columns are empty.<br>
Same for the cancellation code. Cancellations are rare, thus a high number of NaN values for cancellation reasons is okay.

A detailed analysis for these columns is performed in the following.

## Step 6: Predictive Modeling

In [ ]:
df.head()

In [ ]:
# Defining relevant features

In [ ]:
df_pm = df[['DEP_DATETIME_PLANNED', 'ARR_DATETIME_PLANNED', 'AIRLINE_NAME', 'FL_DAYOFWEEK', 'DEP_AIRPORT', 'ARR_AIRPORT', 'DISTANCE', 'DEP_TIME', 'ARR_TIME', 'DEP_DELAY', 'TAXI_OUT_DURATION', 'WHEELS_OFF_TIME', 'WHEELS_ON_TIME', 'IN_AIR_DURATION', 'ARR_DELAY', 'DEP_LAT', 'DEP_LONG', 'DEP_ALT', 'DEP_TZ', 'ARR_LAT', 'ARR_LONG', 'ARR_ALT', 'ARR_TZ']]

In [ ]:
df_pm_sample = df_pm.sample(n=10000)

In [ ]:
y = df_pm_sample 

# Appendix

In [ ]:
df['DIVERTED'].value_counts()

In [ ]:
# Plotting the delay histogram
plt.figure(figsize = (15,6))
plt.hist(df['ARR_DELAY'], bins=1000)
plt.gca().set(title='Arrival Delay Histogram', ylabel='Count');
plt.xlim(-50, 50)
plt.show()

In [ ]:
df[df['DIVERTED'] == 1]

In [ ]:
df[df['DIVERTED'] == 1]

In [ ]:
# All flights of 2018
df[(df['FL_DATE'] >= '2018-01-01') & (df['FL_DATE'] <= '2018-12-31')].head()

<font color='green'><b>bar</b></font>

In [ ]:
#df['ARR_TIME_PLANNED_COPY'] = df['ARR_TIME_PLANNED].apply(lambda x: int((int(x) % 100) + (int(x) - int(x) % 100) * 0.6))

In [ ]:
# Plotting the percentage of delayed flights per year
plt.figure(figsize = (8,6))
zaehler = df_delay[df_delay['ARR_DELAY'] == 1]['FL_YEAR'].value_counts()
nenner = df_delay[df_delay['ARR_DELAY'] == 0]['FL_YEAR'].value_counts()
cat = (zaehler/(zaehler+nenner)).sort_values(ascending=False)
splot=sns.barplot(x=cat.index, y=cat.values, palette='RdYlGn_r')
plt.ylim(0.0, 0.9)
plt.title('Percentage of delayed flights per year', fontsize=16)
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

In [ ]:
x = 1210
print(int(x) % 100)
print((int(x) - int(x) % 100) * 0.6)
print(int(x) % 100 + (int(x) - int(x) % 100) * 0.6)
print(int(str(x)[:-2]) * 60 + int(str(x)[-2:]))

In [ ]:
datetime_str = '09/19/18 13:55:26'
datetime_str2 = '2016-12-31 9:08'
datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(datetime_str2, '%Y-%m-%d %H:%M')
datetime_object2

In [ ]:
df_small = df.iloc[:20]

In [ ]:
df_small.head(5)

In [ ]:
df_small['DEP_TIME'] = df_small['DEP_TIME'].astype('int64')

In [ ]:
df_small.info()

In [ ]:
# Flight delays (2016)
print(len(df[(df['FL_DATE'] >= '2016-01-01') & (df['FL_DATE'] <= '2016-12-31') & (df['ARR_DELAY'] > 0)]))
print(len(df[(df['FL_DATE'] >= '2016-01-01') & (df['FL_DATE'] <= '2016-12-31') & (df['ARR_DELAY'] > 0)]) / len(df[(df['FL_DATE'] >= '2016-01-01') & (df['FL_DATE'] <= '2016-12-31')]))

In [ ]:
df[['DEP_TIME', 'ARR_TIME']].isna().sum() / len(df)

In [ ]:
df['ARR_DELAY'].value_counts().sort_index()

In [ ]:
df['ARR_DELAY'].value_counts().sort_values(ascending=False).iloc[:1]

In [ ]:
# Flight delays (2017)
print(len(df[(df['FL_DATE'] >= '2017-01-01') & (df['FL_DATE'] <= '2017-12-31') & (df['ARR_DELAY'] > 0)]))
print(len(df[(df['FL_DATE'] >= '2017-01-01') & (df['FL_DATE'] <= '2017-12-31') & (df['ARR_DELAY'] > 0)]) / len(df[(df['FL_DATE'] >= '2017-01-01') & (df['FL_DATE'] <= '2017-12-31')]))

In [ ]:
# Flight delays (2018)
print(len(df[(df['FL_DATE'] >= '2018-01-01') & (df['FL_DATE'] <= '2018-12-31') & (df['ARR_DELAY'] > 0)]))
print(len(df[(df['FL_DATE'] >= '2018-01-01') & (df['FL_DATE'] <= '2018-12-31') & (df['ARR_DELAY'] > 0)]) / len(df[(df['FL_DATE'] >= '2018-01-01') & (df['FL_DATE'] <= '2018-12-31')]))

In [ ]:
# Pie chart
labels = ['Not delayed', 'Delayed']
sizes = df_delay[df_delay['FL_YEAR'] == '2016']['ARR_DELAY'].value_counts().values
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  

plt.tight_layout()
plt.title('Percentage of delayed flights in 2016', fontsize=16)
plt.show()